In [1]:
# geometry description (including boundary/mat names)
from ngsolve import * 
from netgen.occ import *
from ngsolve.webgui import * 
# wp = WorkPlane()
# wp.MoveTo(-12,-5).LineTo(-3,-5).NameVertex("fillet")
# wp.LineTo(-3,-1).NameVertex("fillet")
# wp.LineTo(0,-1).LineTo(0,0).LineTo(-12,0).Close()
# geo = wp.Face()
# geo = geo.MakeFillet(list(set(geo.vertices["fillet"])),2)
# geo = geo + geo.Mirror(Axis((0,0,0),X)).Reversed()
# geo = Glue([geo,geo.Mirror(Axis((0,0,0),Y)).Reversed()])
rect = MoveTo(0,0).Rectangle (20,4).Face()
circ = Circle ( (2, 2), 2).Face()
# circ = circ + circ.Mirror(Axis((0,0,0),X)).Reversed()
circ.edges.name="dam"
hole_shape = rect-circ
circ.faces.name="upperlevel"
hole_shape.faces.name="lowerlevel"
geo = Glue([hole_shape, circ])
# geo.faces.Min(X).name="upperlevel"
# geo.faces.Max(X).name="lowerlevel"
rect.edges.name = "wall"
# geo.edges.Nearest((0,0)).name = "dam"
# Draw(geo)
geo = OCCGeometry(geo,dim=2)
# rect = MoveTo(-1,-1).Rectangle (2,2).Face()
# circ = Circle ( (0.7, 0.0), 0.1).Face()
# circ.faces.name="upperlevel"
# hole_shape = rect-circ
# hole_shape.faces.name="lowerlevel"
# shape = Glue([hole_shape, circ])
# mesh = Mesh (OCCGeometry( shape, dim=2).GenerateMesh(maxh=0.3))
# Draw (shape)
# Draw (mesh);

In [2]:
from ngsolve import *
from ngsolve.webgui import Draw
mesh = Mesh(geo.GenerateMesh(maxh=0.5))
#mesh.Curve(3)
Draw(mesh)

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.25…

BaseWebGuiScene

In [3]:
order = 2
fes = L2(mesh,order=order)**3
#fes = L2(mesh,order=order,dim=3)

In [4]:
U,V = fes.TnT() # "Trial" and "Test" function
h, hu, hv = U

# initial conditions
h0mat = {"upperlevel" : 10, "lowerlevel" : 2}
print(1)
U0 = CF((mesh.MaterialCF(h0mat),0,0))
print(0)
# boundary conditions
hbndreg = mesh.BoundaryCF({"wall" : h, "dam" : 0})
hubndreg = mesh.BoundaryCF({"wall" : -hu, "dam" : 0})
hvbndreg = mesh.BoundaryCF({"wall" : -hv, "dam" : 0})

Ubnd = CF((hbndreg,hubndreg,hvbndreg))

# constant for gravitational force
g=1
S0x = 0.03
S0y = 0.0

1
0


In [5]:
h_floor_epsilon = 1e-8
def F(U_cf):
    h_flux, hvx_flux, hvy_flux = U_cf
    h_safe = Max(h_flux, h_floor_epsilon)
    vx = hvx_flux / h_safe
    vy = hvy_flux / h_safe
    
    return CF(((hvx_flux,hvy_flux),
               (hvx_flux*vx + 0.5*g*h_flux**2, hvx_flux*vy),
               (hvx_flux*vy, hvy_flux*vy + 0.5*g*h_flux**2)),dims=(3,2))

In [6]:
n = specialcf.normal(mesh.dim)
def Max(u,v):
    return IfPos(u-v,u,v)
def Max_cf(u_cf,v_cf):
    return IfPos(u_cf-v_cf,u_cf,v_cf)

def Fmax(A_cf,B_cf):
    ha_flux, hua_flux, hva_flux = A_cf
    hb_flux, hub_flux, hvb_flux = B_cf

    ha_safe = Max(ha_flux, h_floor_epsilon)
    hb_safe = Max(hb_flux, h_floor_epsilon)

    vnorma = sqrt(hua_flux**2 + hva_flux**2) / ha_safe
    vnormb = sqrt(hub_flux**2 + hvb_flux**2) / hb_safe
    
    sqrt_gh_a = sqrt(g * Max(ha_flux, 0))
    sqrt_gh_b = sqrt(g * Max(hb_flux, 0))
    
    return Max_cf(vnorma + sqrt_gh_a, vnormb + sqrt_gh_b)


def Fhatn(U_trial):
    Uhat = U_trial.Other(bnd=Ubnd)
    return (0.5*F(U_trial)+0.5*F(Uhat))*n + Fmax(U_trial,Uhat)/2*(U_trial-Uhat)

In [7]:
def DGBilinearForm(fes_arg,F_func,Fhatn_func,Ubnd_cf):
    # fes_arg is the FESpace. U_trial, V_test are from this space.
    U_trial_func, V_test_func = fes_arg.TnT()
    a = BilinearForm(fes_arg, nonassemble=True)
    # Pass U_trial_func to F_func and Fhatn_func
    a += - InnerProduct(F_func(U_trial_func), Grad(V_test_func)) * dx
    a += InnerProduct(Fhatn_func(U_trial_func), V_test_func) * dx(element_boundary=True)
    return a

a = DGBilinearForm(fes,F,Fhatn,Ubnd)

In [8]:
from DGdiffusion import AddArtificialDiffusion

artvisc = Parameter(1.0)
if order > 0:
    AddArtificialDiffusion(a,Ubnd,artvisc,compile=True)

In [9]:
gfu = GridFunction(fes)
gfh, gfhu, gfhv = gfu

gfh_safe_vis = Max(gfh, h_floor_epsilon)
gfvu = gfhu / gfh_safe_vis
gfvv = gfhv / gfh_safe_vis
momentum = CF((gfhu,gfhv))
velocity = CF((gfvu,gfvv))

gfu.Set(U0)
scenes = [ \
    Draw(momentum,mesh,"mom"),
    Draw(velocity,mesh,"vel"),
    Draw(gfh,mesh,"h") ]

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.25…

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.25…

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.25…

In [10]:
def TimeLoop(a_form, current_gfu, dt_val, T_final, nsamplings=100, current_scenes=scenes,
             multidim_draw_flag=False, md_nsamplings=20,
             slope_S0x=0.0, slope_S0y=0.0):
    fes_loop = current_gfu.space # Use space from gfu
    U_trial_loop, V_test_loop = fes_loop.TnT() # For source term assembly

    if multidim_draw_flag:
        gfu_t_loop = GridFunction(fes_loop, multidim=True)

    res_combined = current_gfu.vec.CreateVector() # For M_inv*(K(U) - S_b)
    res_spatial_terms = current_gfu.vec.CreateVector() # For M_inv*K(U)

    t_sim = 0.0; step_idx = 0
    nsteps_total  = int(ceil(T_final/dt_val))    
    
    invma_op = fes_loop.Mass(1).Inverse() @ a_form.mat
    mass_matrix_inverse = fes_loop.Mass(1).Inverse()
    lf_source = LinearForm(fes_loop)

    with TaskManager():
        while t_sim <= T_final - 0.5 * dt_val:
            # 1. Calculate effect of spatial terms: M_inv * K(U^n)
            res_spatial_terms.data = invma_op * current_gfu.vec

            # 2. Calculate effect of bed slope source term: M_inv * S_b(U^n)
            h_current_val = current_gfu.components[0] # This is gfh
            source_b_cf = CF((0, 
                              g * h_current_val * slope_S0x, 
                              g * h_current_val * slope_S0y))
            
            lf_source.Assemble() # Zero out vector
            lf_source += InnerProduct(source_b_cf, V_test_loop) * dx
            current_lf_source_vec = lf_source.Assemble().vec # Assemble and get the vector
            M_inv_S_RHS_vec = mass_matrix_inverse * current_lf_source_vec
            res_combined.data = res_spatial_terms.data - M_inv_S_RHS_vec
            current_gfu.vec.data -= dt_val * res_combined
            
            t_sim += dt_val
            step_idx += 1
            
            # Visualization and printing (original logic)
            if (step_idx % max(1, int(nsteps_total / nsamplings))) == 0:
                for s_scene in current_scenes: s_scene.Redraw()
            if multidim_draw_flag and (step_idx % max(1, int(nsteps_total / md_nsamplings))) == 0:
                gfu_t_loop.AddMultiDimComponent(current_gfu.vec)
            
            print("\rTime = {:.6f} (Step {})".format(t_sim, step_idx), end="")

    print("\nSimulation finished.")
    for s_scene in current_scenes: s_scene.Redraw()
    if multidim_draw_flag:
        return gfu_t_loop

In [ ]:
gfu.Set(U0)
with TaskManager():
    gfu_t = TimeLoop(a, gfu, dt_val=0.001, T_final=5, multidim_draw_flag=True, md_nsamplings=5,
                     slope_S0x=S0x, slope_S0y=S0y)

Draw(gfu_t.components[0],mesh,"h",interpolate_multidim=True,animate=True,
     deformation=True, settings = {"camera" :
                                   {"transformations" :
                                    [{ "type": "rotateX", "angle": -20},
                                     { "type": "rotateZ", "angle": 0}]}},
     min=0, max=12, autoscale=False) 

Time = 4.227000 (Step 4227)

In [15]:
Draw(gfu_t.components[0],mesh,"h",interpolate_multidim=True,animate=True, 
     deformation=True, settings = {"camera" : 
                                   {"transformations" : 
                          
                                    [{ "type": "rotateX", "angle": -20},
                                     { "type": "rotateZ", "angle": 0}]}},
     min=2, max=5, autoscale=False)

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {'camera': {'transformations':…

BaseWebGuiScene